<a href="https://colab.research.google.com/github/arimbawa/Aplikasi-IoT/blob/main/Minggu_7_Keamanan_IoT_(HTTP_%26_MQTT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Minggu ke-7: Keamanan IoT (HTTP & MQTT)**
**Tujuan Pembelajaran:**

1. Mahasiswa memahami risiko keamanan pada komunikasi IoT.
2. Mahasiswa dapat menerapkan HTTPS (SSL/TLS) pada komunikasi HTTP.
3. Mahasiswa dapat mengamankan MQTT dengan username-password authentication.
4. Mahasiswa mengetahui praktik dasar secure coding dalam aplikasi IoT berbasis Flask & MQTT.

# Praktik di Google Colab

# 1. Install modul

In [1]:
!pip install flask pyngrok paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.1 MB/s eta 0:00:00


# 2. Flask dengan Token Authentication

In [2]:
# Konfigurasi Ngrok dengan token Agus
from pyngrok import ngrok

NGROK_TOKEN = "32tnIljunaDDrFMWVCD2784cERi_2tHAN1MwNfRM9Ug5omaq5"
ngrok.set_auth_token(NGROK_TOKEN)

In [3]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

# Token sederhana
API_TOKEN = "iotsecure123"

@app.route('/secure-data', methods=['POST'])
def secure_data():
    token = request.headers.get("Authorization")
    if token != f"Bearer {API_TOKEN}":
        return jsonify({"error": "Unauthorized"}), 401

    data = request.json
    return jsonify({"status": "success", "data_received": data}), 200

# Jalankan ngrok
public_url = ngrok.connect(5000)
print(" * Ngrok URL (secure):", public_url)

app.run(port=5000)

 * Ngrok URL (secure): NgrokTunnel: "https://1588dca5484e.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Oct/2025 02:42:06] "POST /secure-data HTTP/1.1" 401 -
INFO:werkzeug:127.0.0.1 - - [10/Oct/2025 02:42:37] "POST /secure-data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Oct/2025 02:43:39] "POST /secure-data HTTP/1.1" 200 -


**Cara uji:**

In [ ]:
import requests

url = "<NGROK_URL>/secure-data"
headers = {"Authorization": "Bearer iotsecure123"}
data = {"temperature": 25, "humidity": 60}

r = requests.post(url, json=data, headers=headers)
print(r.text)

Jika tanpa header Authorization → 401 Unauthorized.

# 3. MQTT dengan Username & Password (HiveMQ broker publik)

Buat Cluster, dan User Credential terlebih dahulu di Hive MQ Broker

In [6]:
import paho.mqtt.client as mqtt
import time
import json
import random

# MQTT Broker details (using the same as the subscriber)
BROKER = "xxxxxxx.xx.xx.hivemq.cloud" #Temukan ini di informasi cluster yang Anda buat di Hive MQ
PORT = 8883
USERNAME = "xxxx" # Gunakan username dan password user credential yang Anda sudah buat di HiveMQ.
PASSWORD = "xxxxx"
TOPIC = "xxxx"


def on_connect(client, userdata, flags, rc, properties):
    if rc == 0:
        print("Publisher Connected successfully.")
    else:
        print("Publisher Connection failed. Return code:", rc)

# Use callback API version 2
publisher_client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
publisher_client.username_pw_set(USERNAME, PASSWORD)
publisher_client.on_connect = on_connect

# Set TLS/SSL
publisher_client.tls_set()

publisher_client.connect(BROKER, PORT, 60)
publisher_client.loop_start()

try:
    while True:
        temp = round(random.uniform(20, 30), 2)
        hum = round(random.uniform(40, 60), 2)
        data = {"temperature": temp, "humidity": hum}
        payload = json.dumps(data)

        publisher_client.publish(TOPIC, payload)
        print(f"Published: {payload}")
        time.sleep(5) # Publish every 5 seconds

except KeyboardInterrupt:
    print("Stopping publisher.")
    publisher_client.loop_stop()
    publisher_client.disconnect()

Published: {"temperature": 22.24, "humidity": 43.84}
Publisher Connected successfully.
Published: {"temperature": 27.49, "humidity": 42.06}
Published: {"temperature": 24.51, "humidity": 46.13}
Published: {"temperature": 28.66, "humidity": 47.79}
Published: {"temperature": 22.43, "humidity": 52.07}
Published: {"temperature": 27.1, "humidity": 53.34}
Published: {"temperature": 27.56, "humidity": 47.26}
Published: {"temperature": 23.3, "humidity": 46.12}
Stopping publisher.


# Tugas Minggu ke-7

1. Ubah server Flask agar menggunakan HTTPS (bisa dengan sertifikat self-signed atau pyOpenSSL).
2. Buat client IoT yang hanya bisa mengirim data dengan API key valid.
3. Konfigurasi MQTT client agar gagal connect jika username/password salah.
4. Buat laporan singkat: bandingkan komunikasi tanpa keamanan vs dengan keamanan (HTTP/MQTT).T